In [3]:
import numpy as np
import pandas as pd

# This is a section name

## This is a subsection

### Subsubsection
> paragraphs

- list
    - sub.list
* list 2.0
1. numbered list
    1. numbered sub-list

# Markdown commands:
## HTML:
* <p>&#9658</p> UTF-8 shapes
* <a href="https://www.w3schools.com/charsets/ref_utf_geometric.asp">references</a>
<img src="heat100_2.png" alt="valami" title="Kép">

* local image
<img src="https://upload.wikimedia.org/wikipedia/commons/8/87/Double_pendulum_flips_graph.png" width="400">

* image from link
* <i> italics </i>
* <b> bold text </b>

## Markdown/ latex commands:
Use these if you want to save to pdf, as it uses latex converter and ignores HTML input

* $\mathrm{latex}~\mathrm{equation}=\frac{a^2}{\dot{b}}$
* `code style text`
* __bold__ **text**
* _italic_ *text*
***
* horizontal line

## Links
* [Link to section](#this-is-a-section-name)
* <a href="subsubsection">HTML link</a>
* __[External link](https://www.ibm.com/docs/en/watson-studio-local/1.2.3?topic=notebooks-markdown-jupyter-cheatsheet)__
![image](heat100_2.png)
![external image](https://upload.wikimedia.org/wikipedia/commons/8/87/Double_pendulum_flips_graph.png)
* note that `nbconvert` for some reason cannot produce a pdf if you include images from external sources

# Utilities
In the following I will provide some amount of Python code that helps in any kind of data evaluation (or at least I hope it helps).

## Acquiring data
You can use `pandas` to display tables in a fancy way when converting to pdf, or even when importng data from various file formats.

In [4]:
import bs4
from os import PathLike

random_data = np.random.random(size=[2, 7])*100
hdata = np.loadtxt('Hszinkep', dtype=float)
data_panda = pd.read_csv('SDS00002.CSV', skiprows=2, header=None, usecols=[3, 4, 5]).to_numpy()

Now txt and csv is easy, but what about for example `.log` files?

In [9]:

def tablegen(name: str | bytes | PathLike[str] | PathLike[bytes] | int, offset: int | float = 0):
    log = bs4.BeautifulSoup(open(name), 'lxml')
    table = log.table
    rows = table.find_all('row')
    out = []
    t0 = -1
    for tr in rows:
        t = tr.find_all('t')
        t = str([i.text for i in t][0])
        t = sum(x * int(t) for x, t in zip([3600, 60, 1], t.split(":")))
        if t0 == -1: t0 = t+offset
        p = tr.find_all('p')
        try:
            p = float([i.text for i in p][0])
            out.append([t, p])
        except ValueError:
            p = None

    out = np.array(out)
    out[:,0] = out[:,0]-t0
    return np.array([out[:,0], out[:,1]])

log1 = tablegen("VrGauges1.log")

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

Displaying tables, and converting them to latex compatible text:

In [13]:
data = pd.DataFrame(data={'$col_1$': random_data[0], '$col_2$': random_data[1]})
print(data)
print(data.to_latex(column_format='c|c', decimal=',', float_format="$%.2f \pm 0,05$", index=False).replace('\\textbackslash ', '\\').replace('\$', '$').replace('\_', '_'))

     $col_1$    $col_2$
0  20.373640  41.522051
1  62.274995  94.200527
2  98.962062   8.532713
3  89.205366  90.136222
4  65.811935   5.626135
5  84.170606  73.221033
6  59.286774   4.875167
\begin{tabular}{c|c}
\toprule
$col_1$ & $col_2$ \\
\midrule
$20,37 \pm 0,05$ & $41,52 \pm 0,05$ \\
$62,27 \pm 0,05$ & $94,20 \pm 0,05$ \\
$98,96 \pm 0,05$ & $8,53 \pm 0,05$ \\
$89,21 \pm 0,05$ & $90,14 \pm 0,05$ \\
$65,81 \pm 0,05$ & $5,63 \pm 0,05$ \\
$84,17 \pm 0,05$ & $73,22 \pm 0,05$ \\
$59,29 \pm 0,05$ & $4,88 \pm 0,05$ \\
\bottomrule
\end{tabular}


## Making figures
Figures we mostly put together using `matplotlib.pyplot`. It has nearly endless customization options, I'll show some examples of how to use it

In [ ]:
import matplotlib.pyplot as plt
